In [1]:
# Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import Filter

In [10]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint
from scipy.stats import norm

def df_to_array(dataframe):
        # Create Numpy Array
        data_array = dataframe.to_numpy().T

        # Get Column Labels
        labels = dataframe.columns.tolist()

        return data_array, labels

def create_transition_matrix_constraints(n_states):
    # """
    # Creates a constraint that ensures the first 9 parameters (forming a 3x3 matrix) sum to 1 in each row.
    
    # Returns:
    #     constraints: A scipy LinearConstraint object enforcing the row sum constraints.
    # """
    # # Coefficient matrix for the constraint
    # # This matrix is 3x12 because we have 3 constraints (one for each row summing to 1)
    # # and 12 parameters in total, where the first 9 are for the transition matrix.
    # # Each row of coefficients corresponds to a row in the transition matrix.
    # # The last 3 columns are zeros because they correspond to the sigma values, which are not part of the sum.
    # coef_matrix = np.zeros((3, 12))
    # coef_matrix[0, 0:3] = 1  # First row sums to 1
    # coef_matrix[1, 3:6] = 1  # Second row sums to 1
    # coef_matrix[2, 6:9] = 1  # Third row sums to 1
    
    # # The bounds for each constraint are (1, 1), meaning each row must sum exactly to 1.
    # lb = np.ones(3)  # Lower bound of 1 for each constraint
    # ub = np.ones(3)  # Upper bound of 1 for each constraint
    
    # # Create the LinearConstraint object
    # constraints = LinearConstraint(coef_matrix, lb, ub)
    n_params = n_states**2  + n_states # Number of transition matrix parameters
    coef_matrix = np.zeros((n_states, n_params))
    for i in range(n_states):
        coef_matrix[i, i*n_states:(i+1)*n_states] = 1
    lb = np.ones(n_states)  # Lower bound of 1 for each constraint
    ub = np.ones(n_states)  # Upper bound of 1 for each constraint
    return LinearConstraint(coef_matrix, lb, ub)
    
def calculate_initial_probabilities(transition_matrix, n_states):
    """
    Use linalg to find the initial probabilities:

    ! This is not expanded to fit n_states!
    """
    A_matrix = np.vstack(((np.identity(n_states)- transition_matrix), np.ones(n_states)))
    pi_first = np.linalg.inv(A_matrix.T.dot(A_matrix)).dot(A_matrix.T)
    pi_second = np.vstack((np.zeros([n_states,1]), np.ones([1,1])))
    initial_probs = pi_first.dot(pi_second)
    initial_probabilities = initial_probs.T

    return initial_probabilities
def density_function(x, sigma,): #mu, phi):
    # return -0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma[state]) - 0.5 * ((data[t] - mu[state] - phi[state] * data[t-1]) ** 2) / sigma[state] 
    # return np.exp(-0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma[state]) - 0.5 * ((data[t] ) ** 2) / sigma[state]) # - mu[state] - phi[state] * data[t-1]) ** 2) / sigma[state])
    return np.exp(-0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma) - 0.5 * ((x ** 2) / sigma)) # - mu[state] - phi[state] * data[t-1]) ** 2) / sigma[state])

def create_initial_parameters_and_bounds(n_states, data):
    # Initialize transition matrix parameters
    transition_params = np.zeros(n_states**2)
    # Set diagonal elements to 0.95 and distribute the remaining 0.05 evenly among off-diagonal elements in each row
    for i in range(n_states):
        for j in range(n_states):
            if i == j:
                transition_params[i*n_states + j] = 0.95
            else:
                transition_params[i*n_states + j] = (0.05 / (n_states - 1))
    
    base_sigma = np.var(data)
    #sigma_params = [0.5* base_sigma, 2 * base_sigma] 
    sigma_params = [np.sqrt(2 * base_sigma), np.sqrt(0.5 * base_sigma) ]# - n_states / 2  + 0.5 * i for i in range(n_states)]
    # Convert to array for compatibility with optimization functions
    initial_guess = np.array(list(transition_params) + sigma_params)
    
    # Bounds for transition matrix parameters (between 0 and 1)
    transition_bounds = [(0.01, 0.99)] * (n_states**2)
    # Bounds for sigma parameters (greater than 0)
    sigma_bounds = [(0.1, None)] * n_states
    
    return initial_guess, transition_bounds + sigma_bounds

    
def objective_function(params, data, n_states):

    num_obs = len(data)
    transition_matrix, sigma = setup_transition_matrix_and_sigmas(params, n_states)
    transition_matrix = transition_matrix.T

    regularization_strength = 1e-6
    row_sums_penalty = np.sum((transition_matrix.sum(axis=1) - 1) ** 2) * regularization_strength
  
    # Calculate the negative log-likelihood
    predicted_probabilities = np.zeros([n_states, num_obs+1])
    filtered_probabilities = np.zeros([n_states, num_obs])
    smoothed_probabilities = np.zeros([n_states, num_obs])
    likelihood_contributions = np.zeros(num_obs)
    # Form Initial Probabilities
    predicted_probabilities[:,0] = calculate_initial_probabilities(transition_matrix, n_states)
    eta = np.zeros(n_states)
    
    filters = np.zeros(n_states)
    for t in range(5):    # Assume data is a 1D array; adjust as necessary for your application
        #eta = for state in range(n_states)]
        for state in range(n_states):
            eta[state] = density_function(data[t], sigma[state])
    
        #print(eta)
    
        # Filtering Step
        filters = predicted_probabilities[:,t] * eta
        # print('first step')
        # print(filters)
        partials =  predicted_probabilities[:,t] * eta
        vector_ones = np.ones(n_states)
        filtered_probabilities[:,t] = filters / (vector_ones.dot(filters))
        # print('second step')
        # print(filtered_probabilities[:,t])
        # Likelihood
        likelihood_contributions[t]= np.log(np.sum(partials))
        
        # Prediction Step
        predicted_probabilities[:,t+1] = filtered_probabilities[:,t].dot(transition_matrix)
        
        # Smoothing Step
        # ...
        # print(f' Likelihood Value :  {likelihood_contributions[t]}')
        # print(f'  Predicted Probability:  {predicted_probabilities[:, t+1]}')
        # print(f' Filtered Probability :  {filtered_probabilities[:,t] }')
        # print(f'Eta  :  {eta}')
        # print(f'Filter:  {filters}')
        # print(f'Partial  :  {partials}')
    # Incorporate the penalty into the objective
    return -np.sum(likelihood_contributions) + row_sums_penalty * 100

def setup_transition_matrix_and_sigmas(parameters, n_states):
    n_params = n_states**2  # Number of transition matrix parameters
    transition_matrix_raw = np.reshape(parameters[:n_params], (n_states, n_states))
    row_sums = transition_matrix_raw.sum(axis=1).reshape(n_states, 1)
    transition_matrix = transition_matrix_raw / row_sums
    sigma = parameters[n_params:]
    return transition_matrix, sigma

def fit(data, n_states):
    # Initial guess for parameters: 9 for transition matrix, 3 for sigmas
    initial_guess, bounds = create_initial_parameters_and_bounds(n_states, data)
    print('Initial Guess')
    print(initial_guess)
    # initial_guess =  np.array([0.95, 0.01, 0.04, 0.02, 0.95, 0.03, 0.01, 0.01, 0.98, 1.0, 2.0, 3.0])
    # Bounds for each parameter
    # bounds = [(0, 1)]*9 + [(0.001, None)]*3
    # Constraint for the transition matrix'
    #options = {'maxiter': 10000, 'disp': True}  # Increase max iterations and display progress
    constraints = create_transition_matrix_constraints(n_states)
    # Minimize the objective function
    result = minimize(objective_function, initial_guess, args=(data,n_states), bounds=bounds, method='L-BFGS-B', constraints=constraints, )#options=options)
    return result

data=np.genfromtxt('SP500.csv', delimiter=',',usecols=np.arange(0,4)) #loading in first 4 columns
y = data[15097:, 3:4]*100 # 100 times log-returns of the S&P 500 index. January 4, 2010 - till end
y=y.T[0,:] #unpacking numpy array
T = len(y) #length of time series
data = y.T
n_states = 1
result = fit(data, n_states)
print("Optimization Result:")
print(result)
trans, sigma = setup_transition_matrix_and_sigmas(result.x, n_states)
trans

Initial Guess
[0.95       1.42122167 0.71061084]


/home/august/.venvs/jupyter/lib/python3.11/site-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


ValueError: shapes (1,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)

In [72]:
print(trans.dot(np.ones(n_states)))

[1. 1.]


In [3]:
sigma

array([0.57506815, 0.57506815])

In [4]:
params =  np.array([0.95, 0.01, 0.04, 0.02, 0.95, 0.03, 0.01, 0.01, 0.98, 1.0, 2.0, 3.0])

In [5]:
trans2, sigma2 = setup_transition_matrix_and_sigmas(params)
trans2

TypeError: setup_transition_matrix_and_sigmas() missing 1 required positional argument: 'n_states'

In [ ]:

# # Example usage:
# constraints = create_transition_matrix_constraints()

# def setup_transition_matrix_and_sigmas(parameters):
#     """
#     Sets up a transition matrix and sigma values from a flat array of parameters.

#     Parameters:
#     - parameters: A flat array of 12 parameters, where the first 9 are used to create a
#       normalized transition matrix, and the last 3 are sigma values for a Gaussian distribution.

#     Returns:
#     - transition_matrix: A 3x3 stochastic matrix where each row sums to 1.
#     - sigmas: An array of 3 sigma values.
#     """

#     # Ensure there are exactly 12 parameters
#     if len(parameters) != 12:
#         raise ValueError("Expected 12 parameters, received {}".format(len(parameters)))

#     # First 9 parameters for the transition matrix
#     transition_matrix_raw = np.reshape(parameters[:9], (3, 3))

#     # Normalize each row to sum to 1
#     row_sums = transition_matrix_raw.sum(axis=1).reshape(3, 1)  # Reshape for broadcasting
#     transition_matrix = transition_matrix_raw / row_sums

#     # Next 3 parameters are the sigmas
#     sigmas = parameters[9:]

#     return transition_matrix, sigmas

# # Example usage:
# parameters = np.array([0.95, 0.01, 0.04, 0.02, 0.95, 0.03, 0.01, 0.01, 0.98, 1.0, 2.0, 3.0])
# transition_matrix, sigmas = setup_transition_matrix_and_sigmas(parameters)

# print("Transition Matrix:")
# print(transition_matrix)
# print("\nSigmas:")
# print(sigmas)

# def objective_function(sigma):
#     """Example objective function to minimize."""
#     # This is a placeholder for your actual objective function,
#     # which would likely involve the Gaussian distribution parameters.
#     return np.sum(sigma**2)

# def minimize_gaussian():
#     # Initial guess for sigma values
#     sigma_initial = np.array([0.5, 0.5, 0.5])

#     # Transition matrix initial guess (3x3 matrix)
#     # Note: This is just an example. You'll need to adjust it according to your problem.
#     transition_matrix_initial = np.array([[0.33, 0.33, 0.34],
#                                            [0.33, 0.33, 0.34],
#                                            [0.33, 0.33, 0.34]])

#     # Constraint: Rows of the transition matrix sum to 1
#     # This creates 3 constraints, one for each row of the transition matrix.
#     constraints = [LinearConstraint(np.ones((1, 3)), 1, 1) for _ in range(3)]

#     # Perform the minimization, assuming the objective function and constraints are separable
#     # and focusing on the sigma parameter only for simplicity.
#     result = minimize(objective_function, sigma_initial, constraints=constraints)

#     return result

# # Run the minimization process
# result = minimize_gaussian()

# print("Optimization Result:")
# print(result)


In [ ]:
# importlib.reload(Filter)
# from Filter import Base

# df = pd.read_csv('22.csv')
# model = Base(df)
# model.univariate_fit()